<a href="https://colab.research.google.com/github/KJanzon/project-nlp-business-case-automated-customers-reviews/blob/main/categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install -q transformers


In [38]:
import sys
sys.path.append('/content/drive/MyDrive/reviews-project')

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
import pandas as pd

In [52]:
import functions
import importlib

importlib.reload(functions)

# Load your CSV file
train_df = functions.load_and_clean("/content/drive/MyDrive/reviews-project/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

test_df = functions.load_and_clean("/content/drive/MyDrive/reviews-project/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")

# Merge them into one DataFrame
combined_df = pd.concat([train_df, test_df], ignore_index=True)


print("Train sentiment distribution:\n", combined_df["sentiment"].value_counts())


Combined shape: (31968, 9)
Train sentiment distribution:
 sentiment
positive    28908
negative     3060
Name: count, dtype: int64


In [55]:
combined_df['product_info'] = combined_df.apply(
    lambda row: f"{row['name']} | {row['categories']} | {row['primaryCategories']}"
    if pd.notna(row['categories']) and pd.notna(row['primaryCategories'])
    else f"{row['name']} | {row['primaryCategories']}",  # fallback if categories is missing
    axis=1
)

combined_df['categories'] = combined_df['categories'].str.replace(r"[\[\]'\"\\]", "", regex=True)


## Generate sematic embeddings

In [56]:
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer

# Load a strong all-purpose model
model = SentenceTransformer('all-MiniLM-L6-v2')  # fast + accurate

# Embed the product_info text
embeddings = model.encode(combined_df['product_info'].tolist(), show_progress_bar=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/999 [00:00<?, ?it/s]

In [57]:
from sklearn.cluster import KMeans

# Let's try 5 clusters for now
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(embeddings)

# Assign to dataframe
combined_df['cluster'] = cluster_labels


In [58]:
for i in range(num_clusters):
    print(f"\nCluster {i} Sample Products:")
    samples = combined_df[combined_df['cluster'] == i]['product_info'].sample(5, random_state=42)
    for product in samples:
        print("•", product)



Cluster 0 Sample Products:
• Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers | eBook Readers,Electronics Features,Walmart for Business,Tablets,See more Amazon Kindle Voyage (Wi-Fi),Electronics,Amazon Ereaders,Office Electronics,iPad & Tablets,Kindle E-readers,E-Readers & Accessories,All Tablets,See more Amazon Kindle Voyage 4GB, Wi-Fi  3G (Unlocked...,Amazon Book Reader,Computers & Tablets | Electronics
• Kindle E-reader - White, 6 Glare-Free Touchscreen Display, Wi-Fi - Includes Special Offers | Office,eBook Readers,Electronics Features,Walmart for Business,Tablets,Electronics,Amazon Ereaders,Office Electronics,iPad & Tablets,Kindle E-readers,All Tablets,Amazon Book Reader,Computers & Tablets | Office Supplies,Electronics
• Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers | eBook Readers,Electronics Fe

Alternative model

In [60]:
from transformers import pipeline

summarizer = pipeline("summarization", model="google/flan-t5-base")

def summarize_cluster(samples):
    text = " | ".join(samples)
    return summarizer(text[:1000])[0]['summary_text']


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [61]:
import pandas as pd

# Optional: reload your dataset
# df = pd.read_csv("/content/drive/MyDrive/your_project/data/processed/merged_clean_reviews.csv")

# Assume clustering is already done and stored in df['cluster']
# Sample 5 product descriptions per cluster
cluster_samples = {}
for cluster_id in sorted(combined_df['cluster'].unique()):
    samples = combined_df[combined_df['cluster'] == cluster_id]['product_info'].dropna().sample(5, random_state=42).tolist()
    cluster_samples[cluster_id] = samples


In [62]:
def summarize_cluster(samples):
    # Combine samples into one text block, keep it within token limits
    text = " | ".join(samples)
    return summarizer(text[:1000])[0]['summary_text']

# Generate labels for each cluster
cluster_map = {}
for cluster_id, samples in cluster_samples.items():
    print(f"\n🔍 Summarizing Cluster {cluster_id}...")
    summary = summarize_cluster(samples)
    print("Cluster", cluster_id, "→", summary)
    cluster_map[cluster_id] = summary




🔍 Summarizing Cluster 0...
Cluster 0 → Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers | eBook Readers,Electronics Features,Walmart for Business,Tablets,See more

🔍 Summarizing Cluster 1...
Cluster 1 → The Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case | Fire Tablets, Tables, All tablets, Amazon tablets,Computers & Tablets | Electronics | Fire kids edition tablet, 7 display, wi-fi, 16GB, pink Kid-proof case | Fire tablets, learning toys, toys, tablets, amazon tablets, electronics, tech toys, movies, music, computers & tablets, kids' tablets, electronic, iPad & tablet,Tablets for Kids | Toys & Games, electronics | Fire Kids edition tablet , 7 screen, wi - fi, 16g, pink kid-proof case | fire tablets , learning toys, toys , tablet, amazon tablet s, electronics & electronics, kids ' tablets s , electronic , tv , music , children 's tablets, computers

🔍 Summarizing Cluster 

In [70]:
def assign_category(text):
    text = text.lower()

    name_section = text.split('|')[0]  # everything before the first '|'

    accessories_keywords = ['charger', 'power bank', 'usb cable', 'adapter', 'accessory']
    batteries_keywords = ['aa', 'aaa', 'battery', 'batteries', 'alkaline']
    tablet_keywords = ['tablet']
    e_reader_keywords = ['kindle', 'e-reader', 'ereader']
    smart_speaker_keywords = ['echo', 'tap', 'alexa']  # expand with 'tap' and 'alexa'

    if any(kw in name_section for kw in accessories_keywords):
        return "Accessories"
    elif any(kw in name_section for kw in batteries_keywords):
        return "Batteries"
    elif any(kw in name_section for kw in tablet_keywords):
        return "Tablet"
    elif any(kw in name_section for kw in e_reader_keywords):
        return "E-reader"
    elif any(kw in name_section for kw in smart_speaker_keywords):
        return "Smart Speaker"
    else:
        return "Other"



In [71]:
combined_df['product_info_lower'] = combined_df['product_info'].str.lower()
combined_df['product_category'] = combined_df['product_info_lower'].apply(assign_category)

In [72]:
combined_df['product_category'].value_counts()


,count
product_category,
Tablet,17195
Batteries,10858
Smart Speaker,2299
E-reader,1449
Other,88
Accessories,79


In [73]:
categories = combined_df['product_category'].unique()

for category in categories:
    print(f"\n🔷 Category: {category}")
    samples = combined_df[combined_df['product_category'] == category]['product_info'].sample(10, random_state=42)
    for i, product in enumerate(samples, 1):
        print(f"{i}. {product}")



🔷 Category: Batteries
1. AmazonBasics AAA Performance Alkaline Batteries (36 Count) | AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care | Health & Beauty
2. AmazonBasics AAA Performance Alkaline Batteries (36 Count) | AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care | Health & Beauty
3. AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging May Vary | AA,AAA,Electronics Features,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & 